In [1]:
#pip install mysql-connector-python sqlalchemy pandas
import pandas as pd
import sqlalchemy as sal
from sqlalchemy import text

username = 'root'
server_name = 'localhost'
port = '3306'
database_name = 'superstore'

engine = sal.create_engine(f'mysql+mysqlconnector://{username}@{server_name}:{port}/{database_name}')
conn=engine.connect()

In [2]:
#drop table
query = sal.text("drop table product_dim")
p = conn.execute(query)
conn.commit()

In [3]:
#creating a new txt file
products = {'product_id':[100,200,300],'product_name':['iphone 13','HP Laptop Pro','Mac Book'],'price':[50000,60000,90000]}
products_db= pd.DataFrame(products)
products_db.to_csv('products2.txt', index=False)

In [4]:
# #load:
# dfp=pd.DataFrame({'product_key':[],'product_id':[],'product_name':[],'price':[],'start_date':[],'end_date':[]})
# dfp.to_sql('products_dim',con=conn, index=False,if_exists='replace') #or if_exists='append'
# conn.commit()

In [5]:
#create table in mysql
query=sal.text('create table product_dim (product_key int auto_increment primary key, product_id int, product_name varchar(100), price int, start_date date, end_date date)')
p=conn.execute(query)
conn.commit()

In [6]:
# #test
# query = sal.text("INSERT INTO product_dim (product_id, product_name, price, start_date, end_date) VALUES (100, 'abc', 100, '2023-06-01', '9999-12-31')")
# p = conn.execute(query)
# conn.commit()

In [7]:
def extract():
    df_products=pd.read_csv('products2.txt')
    df_products_db=pd.read_sql_query(text('select * from product_dim'),conn)
    return df_products, df_products_db
df_products, df_products_db = extract()

#adding start end date - for testing
df_products['start_date']='2023-06-01'
df_products['end_date']='9999-12-31'
# print(df_products,'\n*********\n',df_products_db)

def transform(df_products,df_products_db):
    df_merged=pd.merge(df_products,df_products_db,how='inner',on='product_id')
    return df_merged

def inserts(df_products):
    df_products.to_sql('product_dim',con=conn, index=False,if_exists='append')
    conn.commit()
inserts(df_products)

df_merged=transform(df_products,df_products_db)
update_rows = df_merged['product_key']
a= update_rows.to_list() #converting series to list
product_keys =','.join([str(key) for key in a])

def updates(product_keys):
    query=sal.text(f'UPDATE product_dim SET end_date = CURRENT_DATE() - INTERVAL 1 DAY WHERE product_key IN ({product_keys})')
    p=conn.execute(query)
    conn.commit()
if product_keys!='':
    updates(product_keys)

In [8]:
#reading the mysql table
df_sql = pd.read_sql_query(text('select * from product_dim'), conn)
df_sql.head(10)

,product_key,product_id,product_name,price,start_date,end_date
0,1,100,iphone 13,50000,2023-06-01,9999-12-31
1,2,200,HP Laptop Pro,60000,2023-06-01,9999-12-31
2,3,300,Mac Book,90000,2023-06-01,9999-12-31


In [9]:
#update changed iphone 13 price and added a new product iphone13 pro
products = {'product_id':[100,200,400],'product_name':['iphone 13','HP Laptop Pro','iphone 13 pro'],'price':[60000,80000,70000]}
products_db= pd.DataFrame(products)
products_db.to_csv('products2.txt', index=False)

In [10]:
#illustration
a=[1,2]
b=[str(key) for key in a]
c= ','.join(b)

In [11]:
def extract():
    df_products=pd.read_csv('products2.txt')
    df_products_db=pd.read_sql_query(text('select * from product_dim'),conn)
    return df_products, df_products_db
df_products, df_products_db = extract()

#adding start end date
df_products['start_date']= pd.to_datetime('now').strftime('%Y-%m-%d')
df_products['end_date']='9999-12-31'

def transform(df_products,df_products_db):
    df_merged=pd.merge(df_products,df_products_db,how='inner',on='product_id')
    return df_merged

def inserts(df_products):
    df_products.to_sql('product_dim',con=conn, index=False,if_exists='append')
    conn.commit()
inserts(df_products)

df_merged=transform(df_products,df_products_db)
update_rows = df_merged['product_key']
a= update_rows.to_list() #converting series to list
product_keys =','.join([str(key) for key in a])

def updates(product_keys):
    query=sal.text(f'UPDATE product_dim SET end_date = CURRENT_DATE() - INTERVAL 1 DAY WHERE product_key IN ({product_keys})')
    p=conn.execute(query)
    conn.commit()
if product_keys!='':
    updates(product_keys)

C:\Users\ragha\AppData\Local\Temp\ipykernel_4052\2225896912.py:8: FutureWarning: The parsing of 'now' in pd.to_datetime without `utc=True` is deprecated. In a future version, this will match Timestamp('now') and Timestamp.now()
  df_products['start_date']= pd.to_datetime('now').strftime('%Y-%m-%d')


In [12]:
#reading the mysql table
df_sql = pd.read_sql_query(text('select * from product_dim'), conn)
df_sql.head(10)

,product_key,product_id,product_name,price,start_date,end_date
0,1,100,iphone 13,50000,2023-06-01,2024-08-19
1,2,200,HP Laptop Pro,60000,2023-06-01,2024-08-19
2,3,300,Mac Book,90000,2023-06-01,9999-12-31
3,4,100,iphone 13,60000,2024-08-20,9999-12-31
4,5,200,HP Laptop Pro,80000,2024-08-20,9999-12-31
5,6,400,iphone 13 pro,70000,2024-08-20,9999-12-31


In [13]:
pd.Timestamp('now').strftime('%Y-%m-%d')

'2024-08-20'

In [14]:
#update
products = {'product_id':[500],'product_name':['Nokia 56'],'price':[5000]}
products_db= pd.DataFrame(products)
products_db.to_csv('products2.txt', index=False)

In [15]:
def extract():
    df_products=pd.read_csv('products2.txt')
    df_products_db=pd.read_sql_query(text('select * from product_dim'),conn)
    return df_products, df_products_db
df_products, df_products_db = extract()

#adding start end date
df_products['start_date']= pd.Timestamp('now').strftime('%Y-%m-%d')
df_products['end_date']='9999-12-31'

def transform(df_products,df_products_db):
    df_merged=pd.merge(df_products,df_products_db,how='inner',on='product_id')
    return df_merged

def inserts(df_products):
    df_products.to_sql('product_dim',con=conn, index=False,if_exists='append')
    conn.commit()
inserts(df_products)

df_merged=transform(df_products,df_products_db)
update_rows = df_merged['product_key']
a= update_rows.to_list() #converting series to list
product_keys =','.join([str(key) for key in a])

def updates(product_keys):
    query=sal.text(f'UPDATE product_dim SET end_date = CURRENT_DATE() - INTERVAL 1 DAY WHERE product_key IN ({product_keys})')
    p=conn.execute(query)
    conn.commit()
if product_keys!='':
    updates(product_keys)

In [16]:
#reading the mysql table
df_sql = pd.read_sql_query(text('select * from product_dim'), conn)
df_sql.head(10)

,product_key,product_id,product_name,price,start_date,end_date
0,1,100,iphone 13,50000,2023-06-01,2024-08-19
1,2,200,HP Laptop Pro,60000,2023-06-01,2024-08-19
2,3,300,Mac Book,90000,2023-06-01,9999-12-31
3,4,100,iphone 13,60000,2024-08-20,9999-12-31
4,5,200,HP Laptop Pro,80000,2024-08-20,9999-12-31
5,6,400,iphone 13 pro,70000,2024-08-20,9999-12-31
6,7,500,Nokia 56,5000,2024-08-20,9999-12-31


In [17]:
#manually changing the date of nokia 56
query=sal.text('update product_dim set start_date = "2023-06-20" where product_key=7')
p=conn.execute(query)
conn.commit()

In [18]:
def extract():
    df_products=pd.read_csv('products2.txt')
    df_products_db=pd.read_sql_query(text('select * from product_dim'),conn)
    return df_products, df_products_db
df_products, df_products_db = extract()

#adding start end date
df_products['start_date']= pd.Timestamp('now').strftime('%Y-%m-%d')
df_products['end_date']='9999-12-31'

def transform(df_products,df_products_db):
    df_merged=pd.merge(df_products,df_products_db,how='inner',on='product_id')
    return df_merged

def inserts(df_products):
    df_products.to_sql('product_dim',con=conn, index=False,if_exists='append')
    conn.commit()
inserts(df_products)

df_merged=transform(df_products,df_products_db)
update_rows = df_merged['product_key']
a= update_rows.to_list() #converting series to list
product_keys =','.join([str(key) for key in a])

def updates(product_keys):
    query=sal.text(f'UPDATE product_dim SET end_date = CURRENT_DATE() - INTERVAL 1 DAY WHERE product_key IN ({product_keys})')
    p=conn.execute(query)
    conn.commit()
if product_keys!='':
    updates(product_keys)

In [19]:
#reading the mysql table
df_sql = pd.read_sql_query(text('select * from product_dim'), conn)
df_sql.head(10)

,product_key,product_id,product_name,price,start_date,end_date
0,1,100,iphone 13,50000,2023-06-01,2024-08-19
1,2,200,HP Laptop Pro,60000,2023-06-01,2024-08-19
2,3,300,Mac Book,90000,2023-06-01,9999-12-31
3,4,100,iphone 13,60000,2024-08-20,9999-12-31
4,5,200,HP Laptop Pro,80000,2024-08-20,9999-12-31
5,6,400,iphone 13 pro,70000,2024-08-20,9999-12-31
6,7,500,Nokia 56,5000,2023-06-20,2024-08-19
7,8,500,Nokia 56,5000,2024-08-20,9999-12-31


In [20]:
#manually changing the date of nokia 56
query=sal.text('update product_dim set end_date = "2023-12-31" where product_key=7')
p=conn.execute(query)
conn.commit()
query=sal.text('update product_dim set start_date = "2024-01-01" where product_key=8')
p=conn.execute(query)
conn.commit()

In [21]:
#reading the mysql table
df_sql = pd.read_sql_query(text('select * from product_dim'), conn)
df_sql.head(10)

,product_key,product_id,product_name,price,start_date,end_date
0,1,100,iphone 13,50000,2023-06-01,2024-08-19
1,2,200,HP Laptop Pro,60000,2023-06-01,2024-08-19
2,3,300,Mac Book,90000,2023-06-01,9999-12-31
3,4,100,iphone 13,60000,2024-08-20,9999-12-31
4,5,200,HP Laptop Pro,80000,2024-08-20,9999-12-31
5,6,400,iphone 13 pro,70000,2024-08-20,9999-12-31
6,7,500,Nokia 56,5000,2023-06-20,2023-12-31
7,8,500,Nokia 56,5000,2024-01-01,9999-12-31


In [22]:
#again updating nokia 56 price
products = {'product_id':[500,400],'product_name':['Nokia 56','iphone 13 pro'],'price':[4000,60000]}
products_db= pd.DataFrame(products)
products_db.to_csv('products2.txt', index=False)

In [23]:
# def extract():
#     df_products=pd.read_csv('products2.txt')
#     df_products_db=pd.read_sql_query(text('select * from product_dim'),conn)
#     return df_products, df_products_db
# df_products, df_products_db = extract()

# #adding start end date
# df_products['start_date']= pd.to_datetime('now').strftime('%Y-%m-%d')
# df_products['end_date']='9999-12-31'

# def transform(df_products,df_products_db):
#     df_merged=pd.merge(df_products,df_products_db,how='inner',on='product_id')
#     return df_merged

# def inserts(df_products):
#     df_products.to_sql('product_dim',con=conn, index=False,if_exists='append')
#     conn.commit()
# inserts(df_products)

# df_merged=transform(df_products,df_products_db)
# update_rows = df_merged['product_key']
# a= update_rows.to_list() #converting series to list
# product_keys =','.join([str(key) for key in a])

# def updates(product_keys):
#     query=sal.text(f'UPDATE product_dim SET end_date = CURRENT_DATE() - INTERVAL 1 DAY WHERE product_key IN ({product_keys})')
#     p=conn.execute(query)
#     conn.commit()
# if product_keys!='':
#     updates(product_keys)

In [24]:
product_keys #gives 2 product_keys but we have to only edit the active record, not the previous record
#Therefor end_date should be high date

'7'

In [25]:
def extract():
    df_products=pd.read_csv('products2.txt')
    df_products_db=pd.read_sql_query(text('select * from product_dim where end_date ="9999-12-31"'),conn)
    return df_products, df_products_db
df_products, df_products_db = extract()

#adding start end date
df_products['start_date']= pd.Timestamp('now').strftime('%Y-%m-%d')
df_products['end_date']='9999-12-31'

def transform(df_products,df_products_db):
    df_merged=pd.merge(df_products,df_products_db,how='inner',on='product_id')
    update_rows= df_merged['product_key']
    k= update_rows.to_list() #converting series to list
    product_keys =','.join([str(key) for key in k])
    return product_keys
product_keys = transform(df_products,df_products_db)

def inserts(df_products):
    df_products.to_sql('product_dim',con=conn, index=False,if_exists='append')
    conn.commit()
inserts(df_products)

def updates(product_keys):
    query=sal.text(f'UPDATE product_dim SET end_date = CURRENT_DATE() - INTERVAL 1 DAY WHERE product_key IN ({product_keys})')
    p=conn.execute(query)
    conn.commit()
if product_keys!='':
    updates(product_keys)

In [26]:
update_rows
product_keys

'8,6'

In [27]:
#reading the mysql table
df_sql = pd.read_sql_query(text('select * from product_dim'), conn)
df_sql.head(10)

,product_key,product_id,product_name,price,start_date,end_date
0,1,100,iphone 13,50000,2023-06-01,2024-08-19
1,2,200,HP Laptop Pro,60000,2023-06-01,2024-08-19
2,3,300,Mac Book,90000,2023-06-01,9999-12-31
3,4,100,iphone 13,60000,2024-08-20,9999-12-31
4,5,200,HP Laptop Pro,80000,2024-08-20,9999-12-31
5,6,400,iphone 13 pro,70000,2024-08-20,2024-08-19
6,7,500,Nokia 56,5000,2023-06-20,2023-12-31
7,8,500,Nokia 56,5000,2024-01-01,2024-08-19
8,9,500,Nokia 56,4000,2024-08-20,9999-12-31
9,10,400,iphone 13 pro,60000,2024-08-20,9999-12-31


In [28]:
#insert time - in case of dupicates in the .txt file. then we can take latest one or group by to take max/min.

In [30]:
sal.__version__ #!pip --upgrade sqlalchemy --user

'2.0.32'